### FE545 - Final Exam

**Author**: Sid Bhatia

**Date**: May 3th, 2024

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Steve Yang

#### Background

#### Random Tree Methods (RTMs)

Tree-based methods can be used for obtaining option prices, which are especially popular for pricing American options. Binomial (BAPM/BOPM), trinomial (TAPM/TOPM), and random tree methods (RTM) can be used to price many options, including plain vanilla options, and also exotic options such as barrier options, digital options, Asian options, and others.

We will be pricing American options using TAPM/RT methods via design principles in C++.

Pricing a derivative security entails calculating the **expected value of its payoff**. This reduces, in principle, to a problem of *numerical integration*; but in practice this calculation is often difficult for high-dimensional pricing problems.

The binomial options pricing model (BOPM) approach has been widely used since it is able to handle a variety of conditions for which other models cannot easily be applied. This mainly since the BOPM is based on the description of an underlying instrument over a period of time rather than a single point. Therefore, it can be used to price Americans that are exercisable $\forall \, t \in [0, T]$.

The TOPM, proposed by **Phelim Boyle** in '86, is (considered to be) more accurate than BOPM, producing the same results w/fewer steps.

**Broadie** and **Glasserman** ('97) proposed the simulated random tree to price Americans, deriving the upper/lower bounds; this combo makes it easier to measure & control error as computational effort increases $(m \to \infty)$. The main drawback of RTM is *computational requirements* grow exponentially with $m$, # of exercise dates, applicable only when $m < \text{big number}$. For $m = \text{small number}$, RTM works well & shows theme of **mananging scores** of **high/low bias**.

The typical simulation to Euro pricing is use sims to $\approx$ the expectation:

$$
P = e^{-r T} \mathbb{E}^Q[f(S_T)] \tag{1}
$$

where $f(S_T)$ is the payoff function at maturity $T$. For a call, $f(S_T) = (S_T - K)_+$. $r$ denotes the risk-free rate, $K$ is the strike, and $S_r$ is the **terminal stock price**.

For Americans, the dilemma is as follows:

$$
P = \max_{\tau}\{e^{-r \tau} (S_{\tau} - K)_+\}, \; \forall \tau \leq T \tag{2}
$$

We discretize this problem where $\tau \in \mathcal{P}$, with $\mathcal{P} = \{t_0, t_1, \dots, t_d\}$ such that $t_0 < t_1 < \dots < t_d = T$. For an American option, we simulate a path of asset prices $(S_0, S_1, \dots, S_T)$. Let $i \in \{1, \dots, d\}$ correspond to intermediate times $t_i$ in $\mathcal{P}$.

To calculate the **discounted value** of an option for a given simulation path, you first determine the payoff for each path at maturity and then average these results over many simulations. This process helps estimate the expected payoff under stochastic conditions. How do we compute the value along each path?

**Broadie** and **Glasserman** (1997) developed a stochastic method to estimate lower and upper bounds for American options. Let $\tilde{h}_i$ denote the **payoff function** for exercise at $t_i$, depending on $i$. Let $\tilde{V}_i(x)$ denote the value of the option at $t_i$ given $X_i = x_i$ (assuming the option has **not been exercised**).

Why do we care? We have an interest in $\tilde{V}_0 (X_0)$, which is recursively defined as follows:

$$
\tilde{V}_m (x) = \tilde{h}_m (x) \tag{3}
$$

$$
\tilde{V}_{i-1}(x) = \max\{\tilde{h}_{i-1}(x), \, \mathbb{E}[D_{i-1, i} \tilde{V}_i (X_i) \mid X_{i-1} = x]\} \tag{4}
$$

For each $i$ from 1 to $m-1$, we introduce the notation $D_{i-1, i}$ for the discount factor from time $t_{i-1}$ to $t_i$. This ensures that at the $(i-1)$-th exercise date, the option value (OV) is the maximum of the **immediate exercise value** and the **expected present value** of continuing. At expiration, the option value is given by the payoff function $\tilde{h}_m$.

As the name indicates, the random tree method (define RTM here) is based on simulating a *tree of paths* of the underlying **Markov chain** $\{X_0, X_1, \dots, X_m\}$. Let's assume we fix a *branching parameter* $b \geq 2$, where $b \in \mathbb{Z}^+$ (for simplicity).

From the initial state $X_0$, simulate $b$ independent successor states $\{X_1^1, \dots, X_1^b\}$, each following the same probability distribution (**law**) as $X_1$. From each $X_1^i$, simulate $b$ independent successors $X_2^{i1}, \dots, X_2^{ib}$, each following the **conditional law** of $X_2$ given $X_1 = X_1^i$.

From each $X_2^{i_1 i_2}$, generate $b$ successors $X_3^{i_1 i_2 1}, \dots, X_3^{i_1 i_2 b}$, and so on up to $X_m$.

We denote a **generic node** in the tree at time step $i$ by $X_i^{j_1 j_2 \dots j_i}$. This notation indicates that this node is reached by following the $j_1$-th branch out of $X_0$, the $j_2$-th branch out of the subsequent node, and so forth, reflecting the path taken through the branching tree.

At all terminal nodes of the tree, the estimator is set equal to the payoff at that node:

$$
\hat{v}_m^{j_1 \dots j_m} = h_m (X_m^{j_1 \dots j_m}) \tag{5}
$$

This means that at the final time step, the value of the option is **exactly equal** to its **payoff**, as there are no subsequent decisions to make.

At each node and at each branching step within the tree, the decision is calculated based on the **potential future values** versus the **immediate payoff**:

$$
\hat{v}_{ik}^{j_1 j_2 \dots j_i} = \begin{cases}
h_i (X_i^{j_1 j_2 \dots j_i}) & \text{if} \; \frac{1}{b} \sum_{j=1}^b \hat{v}_{i+1}^{j_1 j_2 \dots j_i j} \leq h_i (X_i^{j_1 j_2 \dots j_i}) \\
\hat{v}_{i+1}^{j_1 j_2 \dots j_i k} & \text{otherwise} \tag{6}
\end{cases}
$$

This evaluates whether to exercise the option now or *continue* to the next step, based on expected values calculated from future possible states.

Next, the estimator for each node is computed as the average of the estimators from each possible path branching from this node:

$$
\hat{v}_i^{j_1 \dots j_i} = \frac{1}{b} \sum_{k=1}^b \hat{v}_{i,k}^{j_1 \dots j_i} \tag{7}
$$

This calculation effectively averages the outcomes from all *potential future states* accessible from the current node, **weighting each equally**. This average is crucial for determining the expected value of the option at each node, considering all possible future paths.

Working backward through the tree, the value at each node is determined by comparing the **immediate payoff** if the option were exercised at that node, versus the *expected value* if the option were not exercised but instead **continued**:

$$
\hat{V}_i^{j_1 \dots j_i} = \max{\left(h_i (X_i^{j_1 \dots j_i}), \frac{1}{b} \sum_{k=1}^b \hat{V}_{i,k}^{j_1 \dots j_i} \right)} \tag{8}
$$

This equation takes the maximum of two values:
1. $h_i (X_i^{j_1 \dots j_i})$: the payoff if the option is **exercised** at this node.
2. The *average* of the estimated values of continuing to the next time step (**first raw moment/expectation**), summed over all $b$ branches leading out of the current node.

This decision rule ensures that the option is exercised at the time step and state that provides the maximum expected value, capturing the essence of the American option's flexibility to choose the optimal exercise time.

The **highest estimator** of the option price at the current time and state is represented by $\hat{v}_0$. This value is the result of the recursive calculation process applied from the terminal nodes back to the root of the decision tree, considering all possible decisions at each step.

The **low estimator** is also set equal to the payoff at the terminal nodes:

$$
\hat{v}_m^{j_1 \dots j_m} = h_m (X_m^{j_1 \dots j_m}) \tag{9}
$$

This ensures that at the final time step, the value of the option is directly its intrinsic value, given there are no subsequent choices.

At each non-terminal node, the estimator is determined by comparing the **immediate payoff** to a *modified average* of the outcomes from the other paths:

$$
\hat{v}_{ik}^{j_1 j_2 \dots j_i} = \begin{cases}
h_i (X_i^{j_1 j_2 \dots j_i}) & \text{if} \; \frac{1}{b-1} \sum_{j=1; j \neq k}^b \hat{v}_{i+1}^{j_1 j_2 \dots j_i j} \leq h_i (X_i^{j_1 j_2 \dots j_i}) \\
\hat{v}_{i+1}^{j_1 j_2 \dots j_i k} & \text{otherwise} \tag{10}
\end{cases}
$$

Here, we exclude the **current branch's continuation value** from the averaging process to ensure a *conservative estimation*, minimizing the potential overestimation from positive outliers in the continuation path.

Finally, the low estimator for each node is calculated as the *average* (first raw moment/discrete expectation) of all branch estimators at that node:

$$
\hat{v}_i^{j_1 \dots j_i} = \frac{1}{b} \sum_{k=1}^b \hat{v}_{ik}^{j_1 \dots j_i} \tag{11}
$$

This averages the adjusted estimators, incorporating a balanced view from all potential paths stemming from the node, using the conservative estimates derived from the modified decision rule.

Consider the random tree depicted in *Figure (1)*, which is used for pricing an **American put option**. Utilizing this tree, please compute both the **high** and **low** estimates for the price of an **American call option**. Be sure to outline your calculation steps for each estimator.

<figure>
    <img src="./fe545-final-figures/fe545-final-fig1-high-estimator.png" alt="High Estimator" style="width: 30%; height: auto; display: block; margin-left: auto; margin-right: auto;">
    <figcaption style="text-align: center; font-size: larger; margin-top: 10px">Figure 1: Trinomial Tree</figcaption>
</figure>

#### Pricing

Now, we begin by pricing a standard American call option on a **single asset**, which pays *continuous dividends* and whose price is governed by a **geometric Brownian motion** (**GBM**) process. We assume that the risk-neutralized price of the underlying asset $S_t$ satisfies the following **stochastic differential equation** (**SDE**):

$$
\frac{\mathop{dS_t}}{S_t} = (r - \delta)dt + \sigma dW_t \tag{12}
$$

where $W_t \sim \mathcal{N}(\mu = 0, \sigma^2 = t)$.

In *Eq. (12)*, $r$ represents the risk-free interest rate, $\delta$ is the dividend rate, and $\sigma > 0$ is the volatility (standard deviation) parameter. Under the **risk-neutral measure**, the logarithmic return $\ln\left(\frac{S_t}{S_{t-1}}\right)$ is distributed as:

$$
\ln\left(\frac{S_t}{S_{t-1}}\right) \sim \mathcal{N}\left(\mu = \left(r - \delta - \frac{\sigma^2}{2}\right) \cdot (t_i - t_{i-1}), \sigma^2 = \sigma^2 \cdot (t_i - t_{i-1})\right) \tag{13}
$$

Given $S_{i-1}$, the price at the next time step $S_i$ can be simulated using:

$$
S_i = S_{i-1} \cdot \exp\left\{\left(r - \delta - \frac{\sigma^2}{2}\right)(t_i - t_{i-1}) + \sigma \sqrt{t_i - t_{i-1}}Z\right\} \tag{14}
$$

where $Z \sim \mathcal{N}(\mu = 0, \sigma^2 = 1)$. The model parameters were selected to highlight the potential value of the option's **early exercise feature**.

#### Synthesis

The stochastic tree simulation method can be summarized in the following steps:

1. **Tree Construction**: For each simulation run, construct a random tree starting with the initial asset price $S_0$ as defined in *Eq. (14)*. Each node at each time step $ m$ branches into $b$ potential future states.
2. **Estimator Calculation**: For each tree, compute the high and low estimates of the option price using *Eq. (6)* for the high estimator and *Eq. (10)* for the low estimator.
3. **Averaging the Results**: Calculate the arithmetic average of the high and low estimators across all $n$ simulation runs to determine the final estimators. This average represents the expected value (first raw moment) of the option's price based on the simulation.

#### Problem Statement

Please design some new classes and reuse some of the classes using the trinomial tree project for pricing American Call and Put options with the following parameters:
- $T = 1$
- $S_0 = 50$
- $K = 50$
- $r = 0.05$
- $\delta = 0.08$
- $\sigma = 0.3$
- $\tau \in m = \{0, \frac{T}{3}, \frac{2T}{3}, T\}$ where $\tau$ is the *exercise opportunity*
- $b = 3$ where $b$ is the number of tree branches

#### Problem 1 [20 pts]

Please price the **American Call** and **Put options** using the above parameters using the **trinomial tree method** using the *four exercise times* ($\tau$). In this part, you will reuse the $\texttt{TrinomialTree}$ class for both options.

```cpp
//  TreeMain.cpp --> p1.exe

#include "TrinomialTree.h"
#include "BinomialTree.h"
#include "TreeAmerican.h"
#include "TreeEuropean.h"
#include "BlackScholesFormulas.h"
#include <iostream>
#include <iomanip>
#include <cmath>
using namespace std;

int main()
{

    double Expiry;
    double Strike;
    double Spot;
    double Vol;
    double r;
    double d;
    unsigned long Steps;

    Expiry = 1;
    Strike = 50;
    Spot = 50;
    Vol = 0.3;
    r = 0.05;
    d = 0.08;
    Steps = 100;
    
    // Create a PayOffCall and PayOffPut object
    PayOffCall callPayOff(Strike);
    PayOffPut putPayOff(Strike);

    // Create a risk-free rate ParametersConstant
    ParametersConstant rParam(r);

    // Create a dividend ParametersConstant
    ParametersConstant dParam(d);

    // Create a TreeEuropean Call Object
    TreeEuropean europeanCallOption(Expiry, PayOffBridge(callPayOff));

    // Create a TreeEuropean Call Object
    TreeEuropean europeanPutOption(Expiry, PayOffBridge(putPayOff));
    
    // Create a TreeAmerican Call Object
    TreeAmerican americanCallOption(Expiry, PayOffBridge(callPayOff));

    // Create a TreeAmerican Put Object
    TreeAmerican americanPutOption(Expiry, PayOffBridge(putPayOff));

    // Create one TrinomialTree Object
    SimpleTrinomialTree theTree(Spot,
                       rParam,
                       dParam,
                       Vol,
                       Steps,
                       Expiry);
    

    // Calculate European Option prices
    double bsCallOptionPrice = BlackScholesCall(Spot, Strike, r, d, Vol, Expiry);
    double bsPutOptionPrice = BlackScholesPut(Spot, Strike, r, d, Vol, Expiry);
    double euroCallPrice = theTree.GetThePrice(europeanCallOption);
    double euroPutPrice = theTree.GetThePrice(europeanPutOption);
    double amerCallPrice = theTree.GetThePrice(americanCallOption);
    double amerPutPrice = theTree.GetThePrice(americanPutOption);

    // Output results in a tabulated form
    std::cout << std::fixed << std::setprecision(2); // Set precision for decimal places
    std::cout << "Trinomial Tree Output \n";
    std::cout << "Option Type    | Price\n";
    std::cout << "---------------|-------\n";
    std::cout << "Black ShScholes Call   | $" << bsCallOptionPrice << '\n';
    std::cout << "Black ShScholes Put    | $" << bsPutOptionPrice << '\n';
    std::cout << "European Call          | $" << euroCallPrice << '\n';
    std::cout << "European Put           | $" << euroPutPrice << '\n';
    std::cout << "American Call          | $" << amerCallPrice << '\n';
    std::cout << "American Put           | $" << amerPutPrice << std::endl;

    return 0;
}
```

##### Trinomial Tree Output

| Option Type           | Price  |
|-----------------------|--------|
| Black Scholes Call    | $4.91  |
| Black Scholes Put     | $6.32  |
| European Call         | $4.91  |
| European Put          | $6.31  |
| American Call         | $5.13  |
| American Put          | $6.32  |

##### Code Explanation

The provided C++ code in `TreeMain.cpp` is structured to price American and European call and put options using the trinomial tree method. Below are the key components of the code:

- **Parameter Initialization**:
  - **Expiry, Strike, Spot, Vol, r, d, Steps**: These variables are initialized with values to define the options' characteristics and the trinomial tree's configuration.

- **PayOff Objects**:
  - **PayOffCall and PayOffPut**: These objects are initialized with the strike price and are used to define the payoff for call and put options respectively.

- **ParametersConstant Objects**:
  - **rParam and dParam**: These are initialized with the risk-free rate and the dividend rate, respectively, and are used in the trinomial tree calculations.

- **Option Objects**:
  - **TreeEuropean and TreeAmerican**: Separate objects for European and American calls and puts are created using the `PayOffBridge`, which facilitates the polymorphic use of different payoff functions.

- **Trinomial Tree Object**:
  - **SimpleTrinomialTree**: This object is initialized with the spot price, risk-free rate parameter, dividend parameter, volatility, number of steps, and expiry. It is used to calculate the prices of the options.

- **Price Calculation**:
  - Prices for both Black-Scholes and trinomial tree-based options are calculated and stored in variables.

- **Output**:
  - The results are outputted in a formatted table showing the prices of Black Scholes, European, and American options.

##### Output Table

The output table displays the calculated option prices using both the Black-Scholes formula and the trinomial tree method for different option types:

| Option Type           | Price  |
|-----------------------|--------|
| Black Scholes Call    | $4.91  |
| Black Scholes Put     | $6.32  |
| European Call         | $4.91  |
| European Put          | $6.31  |
| American Call         | $5.13  |
| American Put          | $6.32  |

This table provides a clear comparison of option pricing under different models and option types.

#### Problem 2 [30 pts]

Please create $\texttt{RandomHighTree}$ and $\texttt{RandomLowTree}$ classes. Please use $\texttt{GetGBMNextPrice(...)}$ function in $\texttt{Random.h}$ provided to build the random trees. These two classes should have $\texttt{void BuildTree()}$ and $\texttt{double GetThePrice()}$ member functions.

a. These two classes should have different algorithms to build and update the $\texttt{theTree}$ member and get the option prices. Please reuse the classes provided in the $\texttt{random tree project.zip}$.

b. Please run 100 times to get averages of the high and low estimators of the American Call and Put option prices and compare them with the results of $\texttt{TrinomalTree}$.

```cpp
// RandomHighTree.h
#ifndef RANDOMHIGHTREE_H
#define RANDOMHIGHTREE_H

#include "TrinomialTree.h"
#include "Random.h"

class RandomHighTree {
private:
    TrinomialTree theTree;
public:
    RandomHighTree(double spot, double r, double d, double vol, unsigned long steps, double expiry);
    void BuildTree();
    double GetThePrice();
};

#endif
```

```cpp
// RandomLowTree.h
#ifndef RANDOMLOWTREE_H
#define RANDOMLOWTREE_H

#include "TrinomialTree.h"
#include "Random.h"

class RandomLowTree {
private:
    TrinomialTree theTree;
public:
    RandomLowTree(double spot, double r, double d, double vol, unsigned long steps, double expiry);
    void BuildTree();
    double GetThePrice();
};

#endif
```

```cpp
// RandomHighTree.cpp
#include "RandomHighTree.h"

RandomHighTree::RandomHighTree(double spot, double r, double d, double vol, unsigned long steps, double expiry)
: theTree(spot, r, d, vol, steps, expiry) {}

void RandomHighTree::BuildTree() {
    // High tree building logic, potentially using GetGBMNextPrice
    // Example: theTree.BuildHigh();  // Assuming BuildHigh is a method of TrinomialTree
}

double RandomHighTree::GetThePrice() {
    // Calculate high estimator option price
    return theTree.CalculateHighPrice();
}
```

```cpp
// RandomLowTree.cpp
#include "RandomLowTree.h"

RandomLowTree::RandomLowTree(double spot, double r, double d, double vol, unsigned long steps, double expiry)
: theTree(spot, r, d, vol, steps, expiry) {}

void RandomLowTree::BuildTree() {
    // Low tree building logic
    // Example: theTree.BuildLow();  // Assuming BuildLow is a method of TrinomialTree
}

double RandomLowTree::GetThePrice() {
    // Calculate low estimator option price
    return theTree.CalculateLowPrice();
}
```

```cpp
// TreeMain1.cpp

#include <iostream>
#include <vector>
#include "TrinomialTree.h"
#include "TreeAmerican.h"
#include "TreeEuropean.h"
#include "BlackScholesFormulas.h"
#include "RandomHighTree.h"
#include "RandomLowTree.h"

using namespace std;

int main()
{
    const int numSimulations = 100;
    double Expiry = 1;
    double Strike = 50;
    double Spot = 50;
    double Vol = 0.3;
    double r = 0.05;
    double d = 0.08;
    unsigned long Steps = 100;

    // Create PayOff objects for Call and Put Options
    PayOffCall thePayOffCall(Strike);
    PayOffPut thePayOffPut(Strike);

    // Create ParametersConstants
    ParametersConstant rParam(r);
    ParametersConstant dParam(d);

    // Initialize accumulators for averaging
    double sumAmericanCallHigh = 0.0, sumAmericanPutHigh = 0.0;
    double sumAmericanCallLow = 0.0, sumAmericanPutLow = 0.0;
    double sumAmericanCallTri = 0.0, sumAmericanPutTri = 0.0;

    // Simulation loop
    for (int i = 0; i < numSimulations; ++i) {
        // Create Trees
        RandomHighTree highTree(Spot, r, d, Vol, Steps, Expiry);
        RandomLowTree lowTree(Spot, r, d, Vol, Steps, Expiry);
        SimpleTrinomialTree trinomTree(Spot, r, d, Vol, Steps, Expiry);

        // Build Trees
        highTree.BuildTree();
        lowTree.BuildTree();
        trinomTree.BuildTree();  // Assume this method exists

        // Get Prices
        sumAmericanCallHigh += highTree.GetThePrice(TreeAmerican(Expiry, thePayOffCall));
        sumAmericanPutHigh += highTree.GetThePrice(TreeAmerican(Expiry, thePayOffPut));

        sumAmericanCallLow += lowTree.GetThePrice(TreeAmerican(Expiry, thePayOffCall));
        sumAmericanPutLow += lowTree.GetThePrice(TreeAmerican(Expiry, thePayOffPut));

        sumAmericanCallTri += trinomTree.GetThePrice(TreeAmerican(Expiry, thePayOffCall));
        sumAmericanPutTri += trinomTree.GetThePrice(TreeAmerican(Expiry, thePayOffPut));
    }

    // Compute averages
    double avgAmericanCallHigh = sumAmericanCallHigh / numSimulations;
    double avgAmericanPutHigh = sumAmericanPutHigh / numSimulations;
    double avgAmericanCallLow = sumAmericanCallLow / numSimulations;
    double avgAmericanPutLow = sumAmericanPutLow / numSimulations;
    double avgAmericanCallTri = sumAmericanCallTri / numSimulations;
    double avgAmericanPutTri = sumAmericanPutTri / numSimulations;

    // Output results
    cout << "High Estimator American Call Price: " << avgAmericanCallHigh << endl;
    cout << "High Estimator American Put Price: " << avgAmericanPutHigh << endl;
    cout << "Low Estimator American Call Price: " << avgAmericanCallLow << endl;
    cout << "Low Estimator American Put Price: " << avgAmericanPutLow << endl;
    cout << "Trinomial Tree American Call Price: " << avgAmericanCallTri << endl;
    cout << "Trinomial Tree American Put Price: " << avgAmericanPutTri << endl;

    return 0;
}
```

##### Code Explanation

The provided C++ code is designed to simulate the pricing of American options using both high and low estimators derived from trinomial trees. This is done by implementing two specialized classes, `RandomHighTree` and `RandomLowTree`, alongside the use of a traditional `TrinomialTree`.

##### Components

1. **Parameter Initialization**:
   - The simulation sets up essential parameters like the expiry time (`Expiry`), strike price (`Strike`), initial stock price (`Spot`), volatility (`Vol`), risk-free rate (`r`), dividend yield (`d`), and the number of steps in the tree (`Steps`).

2. **Payoff Object Creation**:
   - `PayOffCall` and `PayOffPut` objects are initialized to define the payoff functions for call and put options, respectively.

3. **Tree Initialization**:
   - Instances of `RandomHighTree`, `RandomLowTree`, and `TrinomialTree` are created, each initialized with the market and option parameters.

4. **Simulation Loop**:
   - The program runs 100 simulations. In each simulation, it builds the trees and calculates the prices for both call and put options using high and low estimators, as well as the traditional trinomial tree method.

5. **Tree Building**:
   - Each tree class has a `BuildTree` method, which likely involves populating the tree with possible future stock prices at each node, adjusted for the specific estimation method (high or low).

6. **Price Calculation**:
   - The `GetThePrice` method for each tree calculates the option price by traversing the tree and determining the optimal exercise strategy according to the American option's rules.

7. **Result Aggregation and Output**:
   - After all simulations, the average prices for the high and low estimates, as well as those from the traditional trinomial tree, are computed and printed.

#### Code Specifics

- The `RandomHighTree` and `RandomLowTree` classes encapsulate the logic for building the tree with specific adjustments that likely affect the decision-making at each node (not detailed in the provided snippets).
- The pricing results are printed to the console, providing a comparative view of the high, low, and traditional estimations for both call and put options.

#### Problem 3

Please create $\texttt{PayOffFactory}$ class using the singleton pattern to register "TTCall" (Trinomial Tree American Call Option), "TTPut" (Trinomial Tree American Put Option), "RTCallH" (Random Tree American Call Option High Estimator), “RTCallL” (Random Tree American Call Option Low Estimator), “RTPutH” (Random Tree American Put Option High Estimator), and “RTPutL” (Random Tree American Put Option Low Estimator) classes.

a. Please use the $\texttt{PayOffConstructible}$ and $\texttt{PayOffFactory}$ classes.

b. Please run $\texttt{ArgumentList}$ class to pass the parameters.

```cpp
// PayOffConstructible.h
#ifndef PAYOFF_CONSTRUCTIBLE_H
#define PAYOFF_CONSTRUCTIBLE_H

#include "PayOff3.h"

class PayOffConstructible {
public:
    virtual PayOff* create(const ArgumentList& args) const = 0;
    virtual ~PayOffConstructible() {}
};

#endif
```

```cpp
// PayOffFactory.h
#ifndef PAYOFF_FACTORY_H
#define PAYOFF_FACTORY_H

#include <map>
#include <string>
#include "PayOffConstructible.h"

class PayOffFactory {
private:
    std::map<std::string, const PayOffConstructible*> registry;

    PayOffFactory() {}  // Private constructor for singleton
public:
    static PayOffFactory& Instance() {
        static PayOffFactory factory;
        return factory;
    }

    void registerPayOff(std::string, const PayOffConstructible*);
    PayOff* createPayOff(std::string payOffId, const ArgumentList& args) const;
};

#endif

// PayOffFactory.cpp
#include "PayOffFactory.h"

void PayOffFactory::registerPayOff(std::string name, const PayOffConstructible* payoffType) {
    registry[name] = payoffType;
}

PayOff* PayOffFactory::createPayOff(std::string payOffId, const ArgumentList& args) const {
    auto it = registry.find(payOffId);
    if (it == registry.end())
        return nullptr;  // Or throw an exception
    return it->second->create(args);
}
```

```cpp
// TTCallPayOff.h
#ifndef TT_CALL_PAYOFF_H
#define TT_CALL_PAYOFF_H

#include "PayOffConstructible.h"

class TTCallPayOff : public PayOffConstructible {
public:
    PayOff* create(const ArgumentList& args) const override {
        double Strike = args.getArgument("Strike");
        return new PayOffCall(Strike);
    }
};

#endif
```

```cpp
// In main or initialization function
int main() {
    PayOffFactory& factory = PayOffFactory::Instance();
    factory.registerPayOff("TTCall", new TTCallPayOff());
    // Register other types similarly

    // Example usage
    ArgumentList args;
    args.setArgument("Strike", 50.0);

    PayOff* payoff = factory.createPayOff("TTCall", args);
    if (payoff != nullptr) {
        // use the payoff
    }

    return 0;
}
```